In [2]:
import pandas as pd

In [3]:
# 체크포인트 950
# df = pd.read_csv('/content/evaluation.csv')

# 체크포인트 1900
# df = pd.read_csv('/content/evaluation_ckpt_1900.csv')

# 체크포인트 2580
df = pd.read_csv('/content/evaluation_ckpt_2850.csv')

In [6]:
print(df['고장내용'].loc[0])

3공장렉 철거후 2공장으로 이동


In [4]:
print(df['predictions'].loc[0])

{'고장부품': '3공장 철거', '불량유형': '철거 후 이동', '조치내용': '2공장으로 이동'}


In [5]:
print(df['labels'].loc[0])

{'고장부품': '3공장 렉', '불량유형': '철거', '조치내용': '2공장으로 이동'}


In [7]:
print(df['고장내용'].loc[10])

2공장 3단렉 설치지원


In [8]:
print(df['predictions'].loc[10])

{'고장부품': '2공장 3단렉', '불량유형': '설치 지원', '조치내용': '2공장 3단렉 설치'}


In [9]:
print(df['labels'].loc[10])

{'고장부품': '3단렉', '불량유형': '설치 문제', '조치내용': '2공장에서의 설치 지원'}


## 체크포인트 2850으로 평가

In [26]:
import openai
import json
import pandas as pd
from typing import List, Dict, Union
import ast
from tqdm import tqdm

In [48]:
def evaluate_fault_extraction(
   fault_contents: List[str],
   predictions: List[Union[Dict[str, str], str]],
   labels: List[Union[Dict[str, str], str]],
   client,
   model: str = "gpt-4o"
) -> pd.DataFrame:
   """
   고장내용 추출 결과를 GPT-4로 평가하는 함수
   """

   def parse_dict(item):
       """문자열을 딕셔너리로 변환"""
       if isinstance(item, dict):
           return item
       elif isinstance(item, str):
           try:
               return ast.literal_eval(item)
           except:
               try:
                   return eval(item)
               except:
                   return {}
       else:
           return {}

   evaluation_prompt = """
당신은 산업용 장비 고장 진단 시스템의 성능을 평가하는 전문가입니다.
주어진 고장내용으로부터 '고장부품', '불량유형', '조치내용'을 추출하는 모델의 성능을 평가해야 합니다.

**평가 기준:**

**1. 고장부품 평가 (0-3점)**
- 3점: 정답과 완전히 일치하거나 의미적으로 동등함
- 2점: 핵심 부품명은 정확하지만 불필요한 위치정보나 수식어가 과도하게 포함됨
- 1점: 부품 종류는 맞지만 구체적 식별이 부정확함
- 0점: 완전히 잘못된 부품이거나 누락

**2. 불량유형 평가 (0-3점)**
- 3점: 정답과 완전히 일치하거나 의미적으로 동등함
- 2점: 핵심 불량 성격은 맞지만 문제상황과 지원/조치 행위를 혼동함
- 1점: 불량 카테고리는 맞지만 구체성이 부족하거나 과도함
- 0점: 완전히 잘못된 불량유형이거나 누락

**3. 조치내용 평가 (0-3점)**
- 3점: 정답과 완전히 일치하거나 의미적으로 동등함
- 2점: 핵심 조치 방향은 맞지만 구체성이나 정확성이 떨어짐
- 1점: 조치의 대략적 성격은 맞지만 세부사항이 부정확함
- 0점: 완전히 잘못된 조치내용이거나 누락

**중요 평가 원칙:**
- 부품명에 불필요한 위치정보 포함시 감점
- 문제/고장 상황과 지원/조치 행위 혼동시 감점
- 원문에 없는 정보 추가나 과도한 추론시 감점
- 정보 과다 포함은 누락보다 더 큰 감점 요소
- 실무적 정확성을 최우선으로 평가

마크다운 사용하지 말고 순수한 JSON만 반환하세요:
{"고장부품_점수": 점수, "고장부품_이유": "평가 이유", "불량유형_점수": 점수, "불량유형_이유": "평가 이유", "조치내용_점수": 점수, "조치내용_이유": "평가 이유", "총점": 총점, "종합평가": "전체적인 평가 의견"}
"""

   results = []

   for i, (fault_content, prediction, label) in enumerate(tqdm(zip(fault_contents, predictions, labels), total=len(fault_contents), desc="평가 중")):
       prediction_dict = parse_dict(prediction)
       label_dict = parse_dict(label)

       case_prompt = f"""
**평가 대상:**
고장내용: {fault_content}

**정답 (Label):**
고장부품: {label_dict.get('고장부품', 'N/A')}
불량유형: {label_dict.get('불량유형', 'N/A')}
조치내용: {label_dict.get('조치내용', 'N/A')}

**예측 결과 (Prediction):**
고장부품: {prediction_dict.get('고장부품', 'N/A')}
불량유형: {prediction_dict.get('불량유형', 'N/A')}
조치내용: {prediction_dict.get('조치내용', 'N/A')}

위 기준에 따라 각 항목을 평가하고 순수한 JSON으로만 결과를 반환해주세요.
"""

       response = client.chat.completions.create(
           model=model,
           messages=[
               {"role": "system", "content": evaluation_prompt},
               {"role": "user", "content": case_prompt}
           ],
           temperature=0.1,
           max_tokens=1000
       )

       response_text = response.choices[0].message.content.strip()

       try:
           evaluation_result = json.loads(response_text)
           evaluation_result['index'] = i
           evaluation_result['고장내용'] = fault_content
           evaluation_result['prediction'] = prediction
           evaluation_result['label'] = label
           results.append(evaluation_result)
       except:
           results.append({
               'index': i,
               'error': 'JSON 파싱 실패',
               'raw_response': response_text,
               '고장내용': fault_content,
               'prediction': prediction,
               'label': label
           })

   return pd.DataFrame(results)

In [49]:
# 호출 예시
client = openai.OpenAI(api_key="여러분의 키 값")

In [50]:
fault_contents = df['고장내용'].tolist()[:5]
predictions = df['predictions'].tolist()[:5]
labels = df['labels'].tolist()[:5]

In [51]:
evaluation_results = evaluate_fault_extraction(
   fault_contents=fault_contents,
   predictions=predictions,
   labels=labels,
   client=client
)

print(evaluation_results.head())

평가 중: 100%|██████████| 5/5 [00:20<00:00,  4.03s/it]

   고장부품_점수                                          고장부품_이유  불량유형_점수  \
0        1                  부품 종류는 맞지만 '철거'라는 불필요한 정보가 포함됨.        2   
1        3                                      정답과 완전히 일치함        3   
2        2  핵심 부품명은 정확하지만 '리벳툴' 대신 '리벳'으로 표현되어 구체적 식별이 부정확함        3   
3        0                예측된 '전착도장'은 고장부품이 아니며, 정답은 N/A이다.        0   
4        2                      핵심 부품명은 맞지만 '전후진 솔밸브'가 누락됨.        1   

                                불량유형_이유  조치내용_점수  \
0     핵심 불량 성격은 맞지만 '이동'이라는 조치 행위가 포함됨.        3   
1                           정답과 완전히 일치함        3   
2                           정답과 완전히 일치함        2   
3       예측된 '대차'는 불량유형이 아니며, 정답은 N/A이다.        0   
4  불량 카테고리는 맞지만 '전후진 솔밸브 단선'이 과도하게 포함됨.        2   

                                             조치내용_이유  총점  \
0                                       정답과 완전히 일치함.   6   
1                                      정답과 의미적으로 동등함   9   
2  '리벳 잔영물 제거 청소' 대신 '리벳 잔영물 제거 및 청소'로 표현되어 구체성이 떨어짐   7   
3             

In [52]:
# 총점 출력 코드
print("=== 평가 결과 통계 ===")
print(f"전체 케이스 수: {len(evaluation_results)}")
print(f"고장부품 평균: {evaluation_results['고장부품_점수'].mean():.2f}")
print(f"불량유형 평균: {evaluation_results['불량유형_점수'].mean():.2f}")
print(f"조치내용 평균: {evaluation_results['조치내용_점수'].mean():.2f}")
print(f"총점 평균: {evaluation_results['총점'].mean():.2f}")
print(f"만점(9점) 케이스 수: {len(evaluation_results[evaluation_results['총점'] == 9])}")
print(f"저점수(3점 이하) 케이스 수: {len(evaluation_results[evaluation_results['총점'] <= 3])}")

=== 평가 결과 통계 ===
전체 케이스 수: 5
고장부품 평균: 1.60
불량유형 평균: 1.80
조치내용 평균: 2.00
총점 평균: 5.40
만점(9점) 케이스 수: 1
저점수(3점 이하) 케이스 수: 1


In [53]:
evaluation_results

,고장부품_점수,고장부품_이유,불량유형_점수,불량유형_이유,조치내용_점수,조치내용_이유,총점,종합평가,index,고장내용,prediction,label
0,1,부품 종류는 맞지만 '철거'라는 불필요한 정보가 포함됨.,2,핵심 불량 성격은 맞지만 '이동'이라는 조치 행위가 포함됨.,3,정답과 완전히 일치함.,6,"고장부품과 불량유형에서 불필요한 정보가 포함되어 감점되었으나, 조치내용은 정확하게 ...",0,3공장렉 철거후 2공장으로 이동,"{'고장부품': '3공장 철거', '불량유형': '철거 후 이동', '조치내용': ...","{'고장부품': '3공장 렉', '불량유형': '철거', '조치내용': '2공장으로..."
1,3,정답과 완전히 일치함,3,정답과 완전히 일치함,3,정답과 의미적으로 동등함,9,모든 항목이 정답과 일치하거나 의미적으로 동등하여 높은 점수를 부여함. 예측 결과가...,1,CN7라인 천정 전등 교체,"{'고장부품': 'CN7라인 천정 전등', '불량유형': '고장', '조치내용': ...","{'고장부품': 'CN7라인 천정 전등', '불량유형': '고장', '조치내용': ..."
2,2,핵심 부품명은 정확하지만 '리벳툴' 대신 '리벳'으로 표현되어 구체적 식별이 부정확함,3,정답과 완전히 일치함,2,'리벳 잔영물 제거 청소' 대신 '리벳 잔영물 제거 및 청소'로 표현되어 구체성이 떨어짐,7,"전반적으로 예측이 잘 되었으나, 고장부품과 조치내용에서 약간의 부정확성이 있음. 실...",2,E4L 인너카바 리벳툴 진공이상 리벳 잔영물 제거 청소,"{'고장부품': 'E4L 인너카바 리벳', '불량유형': '진공이상', '조치내용'...","{'고장부품': 'E4L 인너카바 리벳툴', '불량유형': '진공이상', '조치내용..."
3,0,"예측된 '전착도장'은 고장부품이 아니며, 정답은 N/A이다.",0,"예측된 '대차'는 불량유형이 아니며, 정답은 N/A이다.",0,"예측된 '제작 중'은 조치내용이 아니며, 정답은 N/A이다.",0,"모든 항목에서 예측이 정답과 일치하지 않으며, 고장부품, 불량유형, 조치내용 모두 ...",3,전착도장 대차 제작중,"{'고장부품': '전착도장', '불량유형': '대차', '조치내용': '제작 중'}","{'고장부품': '전착도장 대차', '불량유형': nan, '조치내용': '제작중'}"
4,2,핵심 부품명은 맞지만 '전후진 솔밸브'가 누락됨.,1,불량 카테고리는 맞지만 '전후진 솔밸브 단선'이 과도하게 포함됨.,2,핵심 조치 방향은 맞지만 '단선'이라는 구체성이 부족함.,5,"예측 결과는 전반적으로 핵심 요소를 포함하고 있으나, 세부적인 부품명과 불량유형에서...",4,#40 LH 트램스파 동작이상 전후진 솔밸브 단선 수리,"{'고장부품': '#40 LH 트램스파', '불량유형': '동작이상, 전후진 솔밸브...","{'고장부품': '#40 LH 트램스파 전후진 솔밸브', '불량유형': '동작이상'..."


In [54]:
df = pd.read_csv('/content/evaluation.csv')

In [55]:
fault_contents = df['고장내용'].tolist()[:5]
predictions = df['predictions'].tolist()[:5]
labels = df['labels'].tolist()[:5]

In [56]:
evaluation_results = evaluate_fault_extraction(
   fault_contents=fault_contents,
   predictions=predictions,
   labels=labels,
   client=client
)

print(evaluation_results.head())

평가 중: 100%|██████████| 5/5 [00:21<00:00,  4.39s/it]

   고장부품_점수                                     고장부품_이유  불량유형_점수  \
0        3                            정답과 예측이 완전히 일치함.        3   
1        3                                 정답과 완전히 일치함        3   
2        3                          예측 결과가 정답과 완전히 일치함        3   
3        0           전착도장은 고장부품이 아니며, 원문에 고장부품 정보가 없음.        0   
4        2  핵심 부품명인 '#40 LH 트램스파'는 맞지만 '전후진 솔밸브'가 누락됨.        3   

                             불량유형_이유  조치내용_점수  \
0                   정답과 예측이 완전히 일치함.        3   
1                        정답과 완전히 일치함        3   
2                 예측 결과가 정답과 완전히 일치함        2   
3  제작 중은 불량유형이 아니며, 원문에 불량유형 정보가 없음.        0   
4                       정답과 완전히 일치함.        3   

                                   조치내용_이유  총점  \
0                         정답과 예측이 완전히 일치함.   9   
1                            정답과 의미적으로 동등함   9   
2  예측 결과가 정답과 거의 일치하나 '및'이라는 불필요한 연결어가 추가됨   8   
3        제작 중은 조치내용이 아니며, 원문에 조치내용 정보가 없음.   0   
4                             정답과 완전히 일치함.   8   

 

In [57]:
# 총점 출력 코드
print("=== 평가 결과 통계 ===")
print(f"전체 케이스 수: {len(evaluation_results)}")
print(f"고장부품 평균: {evaluation_results['고장부품_점수'].mean():.2f}")
print(f"불량유형 평균: {evaluation_results['불량유형_점수'].mean():.2f}")
print(f"조치내용 평균: {evaluation_results['조치내용_점수'].mean():.2f}")
print(f"총점 평균: {evaluation_results['총점'].mean():.2f}")
print(f"만점(9점) 케이스 수: {len(evaluation_results[evaluation_results['총점'] == 9])}")
print(f"저점수(3점 이하) 케이스 수: {len(evaluation_results[evaluation_results['총점'] <= 3])}")

=== 평가 결과 통계 ===
전체 케이스 수: 5
고장부품 평균: 2.20
불량유형 평균: 2.40
조치내용 평균: 2.20
총점 평균: 6.80
만점(9점) 케이스 수: 2
저점수(3점 이하) 케이스 수: 1


In [58]:
evaluation_results

,고장부품_점수,고장부품_이유,불량유형_점수,불량유형_이유,조치내용_점수,조치내용_이유,총점,종합평가,index,고장내용,prediction,label
0,3,정답과 예측이 완전히 일치함.,3,정답과 예측이 완전히 일치함.,3,정답과 예측이 완전히 일치함.,9,모든 항목에서 정답과 예측이 완전히 일치하여 최고 점수를 부여함.,0,3공장렉 철거후 2공장으로 이동,"{'고장부품': '3공장렉', '불량유형': '철거', '조치내용': '2공장으로 ...","{'고장부품': '3공장 렉', '불량유형': '철거', '조치내용': '2공장으로..."
1,3,정답과 완전히 일치함,3,정답과 완전히 일치함,3,정답과 의미적으로 동등함,9,모든 항목이 정답과 일치하거나 의미적으로 동등하여 높은 점수를 부여함.,1,CN7라인 천정 전등 교체,"{'고장부품': 'CN7라인 천정 전등', '불량유형': '고장', '조치내용': ...","{'고장부품': 'CN7라인 천정 전등', '불량유형': '고장', '조치내용': ..."
2,3,예측 결과가 정답과 완전히 일치함,3,예측 결과가 정답과 완전히 일치함,2,예측 결과가 정답과 거의 일치하나 '및'이라는 불필요한 연결어가 추가됨,8,"전반적으로 예측 결과가 정답과 매우 유사하며, 조치내용에서만 약간의 불필요한 정보가...",2,E4L 인너카바 리벳툴 진공이상 리벳 잔영물 제거 청소,"{'고장부품': 'E4L 인너카바 리벳툴', '불량유형': '진공이상', '조치내용...","{'고장부품': 'E4L 인너카바 리벳툴', '불량유형': '진공이상', '조치내용..."
3,0,"전착도장은 고장부품이 아니며, 원문에 고장부품 정보가 없음.",0,"제작 중은 불량유형이 아니며, 원문에 불량유형 정보가 없음.",0,"제작 중은 조치내용이 아니며, 원문에 조치내용 정보가 없음.",0,"모든 항목에서 원문에 없는 정보를 잘못 추출하여 0점. 원문에 고장부품, 불량유형,...",3,전착도장 대차 제작중,"{'고장부품': '전착도장', '불량유형': '제작 중', '조치내용': '제작 중'}","{'고장부품': '전착도장 대차', '불량유형': nan, '조치내용': '제작중'}"
4,2,핵심 부품명인 '#40 LH 트램스파'는 맞지만 '전후진 솔밸브'가 누락됨.,3,정답과 완전히 일치함.,3,정답과 완전히 일치함.,8,"고장부품에서 일부 정보가 누락되었으나, 불량유형과 조치내용은 정확하게 예측됨. 전반...",4,#40 LH 트램스파 동작이상 전후진 솔밸브 단선 수리,"{'고장부품': '#40 LH 트램스파', '불량유형': '동작이상', '조치내용'...","{'고장부품': '#40 LH 트램스파 전후진 솔밸브', '불량유형': '동작이상'..."
